In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/NASA/apollo_cfd_database.csv"

MessageError: Error: credential propagation was unsuccessful

In [ ]:
df = pd.read_csv(file_path)
print(f"Dataset shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nFirst few rows:")
print(df.head())

In [ ]:
print(f"\n=== DATA CLEANING ===")
print(f"Original dataset size: {len(df):,} points")

# Check for problematic values
negative_theta = df['theta (m)'] < 0
near_zero_re_theta = df['Re-theta'] < 1e-5
print(f"\nProblematic values found:")
print(f"Negative theta values: {negative_theta.sum():,}")
print(f"Near-zero Re-theta values (<1e-5): {near_zero_re_theta.sum():,}")

# Create cleaned dataset
df_clean = df.copy()

# Remove negative theta values (physically impossible)
df_clean = df_clean[df_clean['theta (m)'] >= 0]
print(f"\nAfter removing negative theta: {len(df_clean):,} points")

# Remove extremely small Re-theta values (likely numerical errors)
df_clean = df_clean[df_clean['Re-theta'] >= 1e-5]
print(f"After removing near-zero Re-theta: {len(df_clean):,} points")

# Summary
removed_points = len(df) - len(df_clean)
percent_removed = (removed_points / len(df)) * 100
print(f"\nTotal points removed: {removed_points:,} ({percent_removed:.2f}%)")
print(f"Clean dataset size: {len(df_clean):,} points")

In [ ]:
print(f"\n=== FEATURE ENGINEERING ===")

# NASA's proven baseline features
df_clean['log_density'] = np.log10(df_clean['density (kg/m^3)'])
df_clean['log_velocity'] = np.log10(df_clean['velocity (m/s)'])

# Physics-based features
df_clean['log_dynamic_pressure'] = np.log10(df_clean['dynamic_pressure (Pa)'])
df_clean['velocity_cubed'] = df_clean['velocity (m/s)'] ** 3
df_clean['rho_v_squared'] = df_clean['density (kg/m^3)'] * (df_clean['velocity (m/s)'] ** 2)
df_clean['velocity_squared'] = df_clean['velocity (m/s)'] ** 2

# Spatial features
df_clean['distance_from_center'] = np.sqrt(df_clean['X']**2 + df_clean['Y']**2 + df_clean['Z']**2)

# Normalized spatial coordinates
max_distance = df_clean['distance_from_center'].max()
df_clean['x_normalized'] = df_clean['X'] / max_distance
df_clean['y_normalized'] = df_clean['Y'] / max_distance
df_clean['z_normalized'] = df_clean['Z'] / max_distance

# Input features (14 features)
input_features = [
    'log_density', 'log_velocity', 'aoa (degrees)', 'log_dynamic_pressure',
    'X', 'Y', 'Z', 'distance_from_center', 'x_normalized', 'y_normalized',
    'z_normalized', 'velocity_cubed', 'rho_v_squared', 'velocity_squared'
]

target_variable = 'qw (W/m^2)'

print(f"Input features ({len(input_features)}):")
for i, feature in enumerate(input_features, 1):
    print(f"  {i:2d}. {feature}")
print(f"\nTarget variable: {target_variable}")

In [ ]:
# Column names
VEL = 'velocity (m/s)'
RHO = 'density (kg/m^3)'
AOA = 'aoa (degrees)'

# Unique trajectory states
states = df_clean[[VEL, RHO, AOA]].drop_duplicates().reset_index(drop=True)
print(f"Total unique trajectory states: {len(states)}")

# Step 1: Pick TEST as strictly "in-between" states per AoA
test_keys = set()
neighbor_lock = set()  # Neighbors of test states that must remain TRAIN

for aoa, grp in states.groupby(AOA, sort=False):
    # Physical ordering: velocity ↓ (fast→slow), break ties by density ↑ (thin→thick)
    g = grp.sort_values([VEL, RHO], ascending=[False, True]).reset_index(drop=True)
    n = len(g)
    if n <= 2:
        continue

    mid_idx = np.arange(1, n-1)  # Strictly in-between (no endpoints)
    # Choose ~10% of mids for test (at least 1 if possible), spaced along the sequence
    k = max(1, int(round(0.10 * len(mid_idx))))
    pick_positions = np.linspace(0, len(mid_idx)-1, k, dtype=int)
    picked = mid_idx[pick_positions]

    for i in picked:
        key = tuple(g.loc[i, [VEL, RHO, AOA]])
        left = tuple(g.loc[i-1, [VEL, RHO, AOA]])
        right = tuple(g.loc[i+1, [VEL, RHO, AOA]])
        test_keys.add(key)
        neighbor_lock.add(left)
        neighbor_lock.add(right)

# Step 2: VAL from remaining states (never from neighbor-locked)
remaining = states[~states.apply(lambda r: tuple(r) in test_keys, axis=1)]
remain_locked = remaining[remaining.apply(lambda r: tuple(r) in neighbor_lock, axis=1)]
remain_free = remaining[~remaining.apply(lambda r: tuple(r) in neighbor_lock, axis=1)]

target_val = int(round(0.10 * len(states)))  # ≈10% of all states

# Proportional per-AoA sampling for VAL from the free pool
if len(remain_free) <= target_val:
    val_states = remain_free
else:
    parts = []
    for aoa, grp in remain_free.groupby(AOA, sort=False):
        take = int(round(target_val * len(grp) / len(remain_free)))
        take = min(take, len(grp))
        if take > 0:
            parts.append(grp.sample(n=take, random_state=42))
    val_states = pd.concat(parts).drop_duplicates()
    # Trim or top up slight rounding
    if len(val_states) > target_val:
        val_states = val_states.sample(n=target_val, random_state=42)

train_states = pd.concat([
    remain_locked,
    remain_free[~remain_free.index.isin(val_states.index)]
], axis=0).drop_duplicates()

# Step 3: Map back to full dataframe
train_keys = set(map(tuple, train_states[[VEL, RHO, AOA]].to_numpy()))
val_keys = set(map(tuple, val_states[[VEL, RHO, AOA]].to_numpy()))

def assign_split(row):
    key = (row[VEL], row[RHO], row[AOA])
    if key in test_keys:
        return 'test'
    if key in val_keys:
        return 'val'
    if key in train_keys:
        return 'train'
    return 'other'

df_clean['split'] = df_clean.apply(assign_split, axis=1)

# Step 4: Sanity checks
uniq_counts = (df_clean[[VEL, RHO, AOA, 'split']].drop_duplicates()
               .groupby('split').size())
print("\nUnique states per split:")
print(uniq_counts)

# Verify "in-between" and neighbor-in-train per AoA
violations = []
for aoa, grp in states.groupby(AOA, sort=False):
    g = grp.sort_values([VEL, RHO], ascending=[False, True]).reset_index(drop=True)
    for i in range(1, len(g)-1):
        key = tuple(g.loc[i, [VEL, RHO, AOA]])
        if key in test_keys:
            left = tuple(g.loc[i-1, [VEL, RHO, AOA]])
            right = tuple(g.loc[i+1, [VEL, RHO, AOA]])
            if left not in train_keys or right not in train_keys:
                violations.append((aoa, i, left, key, right))

print(f"In-between violations: {len(violations)}")

# Extract train/val/test data
train_df = df_clean[df_clean['split'] == 'train']
val_df = df_clean[df_clean['split'] == 'val']
test_df = df_clean[df_clean['split'] == 'test']

X_train = train_df[input_features]
y_train = train_df[target_variable]
X_val = val_df[input_features]
y_val = val_df[target_variable]
X_test = test_df[input_features]
y_test = test_df[target_variable]

print(f"\nData splits:")
print(f"Training: {len(X_train):,} points ({len(X_train)/len(df_clean)*100:.1f}%)")
print(f"Validation: {len(X_val):,} points ({len(X_val)/len(df_clean)*100:.1f}%)")
print(f"Test: {len(X_test):,} points ({len(X_test)/len(df_clean)*100:.1f}%)")

In [ ]:
# Use RobustScaler (less sensitive to outliers)
scaler_X = RobustScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)

# Log transform and robust scale targets
y_train_log = np.log10(y_train)
y_val_log = np.log10(y_val)
y_test_log = np.log10(y_test)

scaler_y = RobustScaler()
y_train_scaled = scaler_y.fit_transform(y_train_log.values.reshape(-1, 1)).flatten()
y_val_scaled = scaler_y.transform(y_val_log.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test_log.values.reshape(-1, 1)).flatten()

print(f"RobustScaler applied to {X_train_scaled.shape[1]} input features")
print(f"Target scaling - Log range: {y_train_log.min():.3f} to {y_train_log.max():.3f}")
print("Data scaling complete.")

In [ ]:
def create_improved_model(input_shape):
    """Create improved neural network architecture for heat flux prediction"""
    model = keras.Sequential([
        layers.Input(shape=(input_shape,)),
        layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0001)),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0001)),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0001)),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0001)),
        layers.Dropout(0.2),
        layers.Dense(16, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0001)),
        layers.Dense(1, activation='linear')  # Linear activation for regression
    ])
    return model

# Create model
model = create_improved_model(X_train_scaled.shape[1])

# Configure optimizer
optimizer = keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7
)

model.compile(
    optimizer=optimizer,
    loss='huber',  # More robust to outliers than MSE
    metrics=['mae']
)

print(f"Model architecture: {len(input_features)}→256→128→64→32→16→1")
print(f"Total parameters: {model.count_params():,}")
model.summary()

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.7,
        patience=10,
        min_lr=1e-7,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        'best_heat_flux_model.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
]

print("Starting training...")
history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_val_scaled, y_val_scaled),
    epochs=150,
    batch_size=2048,
    callbacks=callbacks,
    verbose=1
)

print("Training completed!")

In [ ]:
def evaluate_model(y_actual, y_predicted_scaled, scaler_y, model_name="Model"):
    """Comprehensive model evaluation function"""
    # Inverse transform predictions
    y_predicted_log = scaler_y.inverse_transform(y_predicted_scaled.reshape(-1, 1)).flatten()
    y_predicted = 10**y_predicted_log  # Inverse log10 transform

    # Calculate metrics
    mae = mean_absolute_error(y_actual, y_predicted)
    mse = mean_squared_error(y_actual, y_predicted)
    rmse = np.sqrt(mse)

    # Calculate relative error for each point
    relative_errors = np.abs(y_actual - y_predicted) / y_actual * 100

    # Percentage within ±5% error (NASA target)
    percent_within_5 = np.sum(relative_errors <= 5) / len(relative_errors) * 100

    # Median relative error
    median_error = np.median(relative_errors)

    print(f"\n--- {model_name} Evaluation ---")
    print(f"  MAE: {mae:.2f} W/m²")
    print(f"  MSE: {mse:.2f}")
    print(f"  RMSE: {rmse:.2f} W/m²")
    print(f"  Percentage within ±5% error: {percent_within_5:.2f}%")
    print(f"  Median Relative Error: {median_error:.2f}%")
    print("------------------------------")

    return {
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
        'percent_within_5': percent_within_5,
        'median_error': median_error,
        'relative_errors': relative_errors,
        'predictions': y_predicted
    }

# Make predictions and evaluate
print("Making predictions on test set...")
y_pred_scaled = model.predict(X_test_scaled, verbose=0)
results = evaluate_model(y_test, y_pred_scaled, scaler_y, "Neural Network")

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('NASA Heat Flux Prediction: Neural Network Results', fontsize=16)

# 1. NASA target comparison
nasa_target = 95.0
current_accuracy = results['percent_within_5']

models = ['Current Model', 'NASA Target']
accuracies = [current_accuracy, nasa_target]
colors = ['green' if current_accuracy >= nasa_target else 'orange', 'blue']

bars = axes[0,0].bar(models, accuracies, color=colors, alpha=0.7)
axes[0,0].set_ylabel('% Within ±5%')
axes[0,0].set_title('Performance vs NASA Target')
axes[0,0].set_ylim(0, 100)

# Add value labels
for bar, acc in zip(bars, accuracies):
    axes[0,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                   f'{acc:.1f}%', ha='center', va='bottom', fontweight='bold')

# 2. Predicted vs Actual (log scale)
axes[0,1].scatter(y_test, results['predictions'], alpha=0.1, s=0.5, color='purple')
axes[0,1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', alpha=0.8)
axes[0,1].set_xlabel('Actual Heat Flux (W/m²)')
axes[0,1].set_ylabel('Predicted Heat Flux (W/m²)')
axes[0,1].set_title('Predicted vs Actual Heat Flux')
axes[0,1].set_xscale('log')
axes[0,1].set_yscale('log')

# 3. Relative error distribution
axes[0,2].hist(results['relative_errors'], bins=50, alpha=0.7, color='purple', edgecolor='black')
axes[0,2].axvline(5, color='red', linestyle='--', label='NASA ±5% Target')
axes[0,2].axvline(results['median_error'], color='green', linestyle='-',
                  label=f'Median: {results["median_error"]:.1f}%')
axes[0,2].set_xlabel('Relative Error (%)')
axes[0,2].set_ylabel('Frequency')
axes[0,2].set_title('Relative Error Distribution')
axes[0,2].legend()
axes[0,2].set_xlim(0, 25)

# 4. Training history
axes[1,0].plot(history.history['loss'], label='Training Loss', alpha=0.8)
axes[1,0].plot(history.history['val_loss'], label='Validation Loss', alpha=0.8)
axes[1,0].set_xlabel('Epoch')
axes[1,0].set_ylabel('Loss (Huber)')
axes[1,0].set_title('Training History')
axes[1,0].legend()
axes[1,0].set_yscale('log')

# 5. Error vs heat flux magnitude
axes[1,1].scatter(y_test, results['relative_errors'], alpha=0.1, s=0.5, color='purple')
axes[1,1].axhline(5, color='red', linestyle='--', label='NASA ±5% Target')
axes[1,1].set_xlabel('Actual Heat Flux (W/m²)')
axes[1,1].set_ylabel('Relative Error (%)')
axes[1,1].set_title('Error vs Heat Flux Magnitude')
axes[1,1].set_xscale('log')
axes[1,1].legend()
axes[1,1].set_ylim(0, 50)

# 6. Error breakdown pie chart
within_5 = results['percent_within_5']
within_5_10 = np.sum((results['relative_errors'] > 5) & (results['relative_errors'] <= 10)) / len(results['relative_errors']) * 100
above_10 = 100 - within_5 - within_5_10

labels = ['Within ±5%\n(NASA Target)', 'Within ±5-10%', 'Above ±10%']
sizes = [within_5, within_5_10, above_10]
colors_pie = ['green', 'yellow', 'red']
explode = (0.1, 0, 0)

axes[1,2].pie(sizes, explode=explode, labels=labels, colors=colors_pie,
              autopct='%1.1f%%', shadow=True)
axes[1,2].set_title('Error Distribution Breakdown')

plt.tight_layout()
plt.show()

In [ ]:
print(f"\nMODEL CONFIGURATION:")
print(f"   Architecture: {len(input_features)}→256→128→64→32→16→1")
print(f"   Parameters: {model.count_params():,}")
print(f"   Training epochs: {len(history.history['loss'])} (early stopped)")

print(f"\nPERFORMANCE RESULTS:")
print(f"   Current model: {results['percent_within_5']:.1f}% within ±5%")
print(f"   NASA target: 95.0% within ±5%")
print(f"   Median error: {results['median_error']:.1f}%")

if results['percent_within_5'] >= 95:
    print(f"   SUCCESS: Beat NASA's interpolation baseline!")
else:
    remaining = 95 - results['percent_within_5']
    print(f"   Progress: {remaining:.1f}% improvement needed to reach NASA baseline")

print(f"\nAnalysis complete!")